In [1]:
from transformers import pipeline  # loding prebuild nlp model like sentiment analysis
from sklearn.feature_extraction.text import TfidfVectorizer # for keyword extraction
import json # printing the profile and readable format

In [2]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True) # function that accepts text and returns sentiment label

Device set to use cpu
C:\Users\jeesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [3]:
conversation_log = [] # stores all user inputs
personality_profile = { # profile dictionary
    "emotion_counts": {}, # stors the words representing emotions
    "frequent_words": [], # top frequent words from user input
    "mental_state": "Neutral" # overall mental and emotional state
}

In [16]:
def process_input(user_input): # function to get user input
    results = classifier(user_input)[0] # gets the emotion results from the classifier
    #total_score = sum([r['score'] for r in results]) # sum of all emotional scores
    top_emotion = max(results, key=lambda x: x['score']) # choose the emotion with highest confidence
    emotion_label = top_emotion['label'] # extracts the emotions
    emotion_score = top_emotion['score'] # extarcts score
    #emotion_percentage = ((emotion_score / total_score)*100) # converting the score of dominant emotion to percentage
    conversation_log.append(user_input) # saves conversation to memory
    # counts amount of times a particular emotion appears
    if emotion_label in personality_profile["emotion_counts"]:
        personality_profile["emotion_counts"][emotion_label] +=1 # increments by one 
    else:
        personality_profile["emotion_counts"][emotion_label] =1 # initialize count
    
    # map emotion to mental state
    emotion_to_state ={
        "joy": "Excited",
        "love": "Stable",
        "anger": "Frustrated",
        "fear": "Anxious",
        "sadness": "Depressed",
        "surprise": "Stressed",
        "neutral": "Neutral"
    }
    mental_state = emotion_to_state.get(emotion_label.lower(),"Unclear")
    # now we convert emotions to lower case and map to state
    personality_profile["mental_state"] = mental_state
    analyze_frequent_words() # update frequent words
    result = {
        "dominanr_emotion": emotion_label, 
        "emotion_score": round(emotion_score,3), # returns value between 0 and 1
        "mental_state": mental_state
    }
    print(f"\n {result}") # see result values
    return result

In [17]:
def analyze_frequent_words(): # function to analyse the frequency of words
    if conversation_log:
        vectorizer = TfidfVectorizer(stop_words='english') # remove less meaningful or regular words
        X = vectorizer.fit_transform(conversation_log) # fits model to all past user input
        words = vectorizer.get_feature_names_out() # extract words
        scores = X.sum(axis=0).A1 # stores the score for each words
        top_words = [words[i] for i in scores.argsort()[::-1][:5]] # extracts top 5 keywords
        personality_profile["frequent_words"] = top_words # updates profile with the most frequent words

In [18]:
def show_profile():
    analyze_frequent_words() # this line calls the above functions
    print(json.dumps(personality_profile, indent = 4)) # prints the profile

In [1]:
# print("you can talk to me and type 'exit' to stop")
# while True:
#     user_input = input("You: ") # user input
#     if user_input.lower() =="exit":
#         break
#     process_input(user_input) # process and learn from it
#     emotion_data = personality_profile["emotion_counts"] # shows the emotion data
#     if emotion_data:
#         current_emotion = max(emotion_data, key=emotion_data.get)
#         current_state = personality_profile["mental_state"]
#         print(f"detected emotions: {current_emotion} | Estimated mental state: {current_state}\n")
#     analyze_frequent_words() # shows the updated frequent word after everyinput
#     print(f"frequent words so far: {personality_profile['frequent_words']}\n")

In [11]:
show_profile()

{
    "emotion_counts": {
        "neutral": 4,
        "anger": 1,
        "joy": 5,
        "sadness": 4,
        "surprise": 2
    },
    "mental_state": "Excited",
    "frequent_words": [
        "ok",
        "today",
        "ooh",
        "nothappy",
        "holla"
    ]
}
